In [ ]:
#觸發事件:AWS_eventBride(cloudWatchEvents)

import json
import requests
import boto3

AWS_LAMBDA_TELEGRAM_ADMINBOT_URL = "此為觸發telegram bot lambda的API Gateway URL"

def sendTelegramMessage(msg):
    try:
        with requests.Session() as s:
            res = s.post(
                AWS_LAMBDA_TELEGRAM_ADMINBOT_URL,
                data = json.dumps({"msg":msg}))
            print(res.content)
    except Exception as e:
        app.logger.error(e)

def lambda_handler(event, context):
    # TODO implement
    print(event) #會在cloudwatch log中打印出來
    
    detail_type = event["detail-type"]
    
    if "State Change" in detail_type:
        time = event["time"]
        referenceType = event["detail"]["referenceType"]
        repositoryName = event["detail"]["repositoryName"]
        referenceName = event["detail"]["referenceName"]
        eventName = event["detail"]["event"]
        commitID = event["detail"]["commitId"] #獲取後用來提取詳細資料(修改者名稱 & git commit時的備註訊息)

        #利用boto3的codecommit服務去獲取所獲得commitID詳細資訊
        cct = boto3.client('codecommit', region_name="us-west-2", aws_access_key_id='A*************J3', aws_secret_access_key='3C9************L+')
        response = cct.get_commit(
            repositoryName='Ga***********ess',
            commitId=commitID
        )
        account = response['commit']['author']['name']
        message = response['commit']['message']

        #telegram發送最終訊息
        teleMsg = f"<AWS_CodeCommit變更> \n修改者:{account} \n變更事件:{eventName} \n變更時間:{time} \n倉倉庫名稱:{repositoryName} \n分支名稱:{referenceName} \n備註訊息:{message}"
        sendTelegramMessage(teleMsg)
        
    return {
        'statusCode': 200,
        'body': json.dumps(event)
    }